In [1]:
import matplotlib
%matplotlib tk
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%autosave 180

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import sys
sys.path.append('..')
from visualize import visualize_svm as Visualize

from tqdm import trange, tqdm

sys.path.append('../..')
from svm import svm as SVM

import scipy
from statsmodels.stats.multitest import multipletests

#
import pandas as pd
import matplotlib.patches as mpatches
import scipy.stats as stats


Autosaving every 180 seconds


<ipython-input-1-2ad95b854a16>:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [208]:
# #######################################################
# #### RERUN SVM BUT LOWPASS FILTER LOCANMF COMPS #######
# #######################################################

# #



def load_locaNMF_data_lockout(animal_id, window_length):
    # order locaNMF components by plot color ORDER in Fig 4A

    # FIRST LOAD AREA NAMES FROM A RANDOM FILE
    data = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Mar31_30Hz/IA1pm_Mar31_30Hz_locanmf.npz',
               allow_pickle=True)

    names = data['names']
    #for k in range(len(names)):
    #    print (k, names[k])

    #######################################
    #######################################
    #######################################
    root_dir = '/media/cat/4TBSSD/yuki/'
    fname = os.path.join(root_dir,
                         animal_id,
                         'super_sessions',
                         'alldata_body_and_nonreward_lockout_'+str(lockout)+'secLockout_[]bodyfeats.npz')
        
    data = np.load(fname,
                   allow_pickle=True)

    locaNMF_temporal = data['trials']

    locaNMF_temporal = np.vstack(locaNMF_temporal)
    print (locaNMF_temporal.shape)
    
    # Resplit the locaNMF components into 50 session chunks
    #idx = np.arange(0,locaNMF_temporal.shape[0],50)[1:]
    #locaNMF_temporal = np.array_split(locaNMF_temporal,idx)
    
    #
    temp_out = []
    for k in range(0, len(locaNMF_temporal), window_length):
        temp = locaNMF_temporal[k:k+200]
        if temp.shape[0]<window_length:
            continue
       # print ("locanmf split into 50 chunk segs: ", temp.shape)
        temp_out.append(temp)
        
    locaNMF_temporal = np.array(temp_out)
  
    return locaNMF_temporal, names

#
def compute_variance_locaNMF(locaNMF_temporal):

    # 
    means = []
    var = []
    for k in range(len(locaNMF_temporal)):
        temp = locaNMF_temporal[k].mean(0)
        #temp = np.median(locaNMF_temporal[k], axis=0)
        means.append(temp)

        # 
        #print ("computing variance")
        temp2 = locaNMF_temporal[k]
        print ('temp: ', temp.shape)
        
        l = []
        for k in range(temp2.shape[2]):
            temp3 = np.var(temp2[:,:,k],axis=0)
            l.append(temp3)
        #print ('temp out: ', temp.shape)
        
        var.append(np.array(l))

    #
    means = np.array(means)
    var = np.array(var)
    
    return means, var

#
def load_variances(animal_id, lockout, window_length):

    #

    # 
    locaNMF_temporal, names = load_locaNMF_data_lockout(animal_id, 
                                                       window_length)

    # 
    means, var = compute_variance_locaNMF(locaNMF_temporal)

    #

    return means, var, names


def find_first_variance_decrease_point2(data_in, s1, e1, std_factor, ctr,
                                        animal_id,
                                        n_vals_below_thresh=30):
    
    # 
    if False:
    #if ctr!=3:
        data_in = savgol_filter(data_in, 31, 2)

    # find std of up to 10 sec prior to pull
    std = np.std(data_in[s1:e1], axis=0)

    # find mean up to 10 sec prior to pull
    mean2 = np.mean(data_in[0:e1], axis=0)

    # do rolling evalution to find location when next N vals are belw threhsold
    idx_out = np.nan
    #n_vals_below_thresh = 30
    window = [20*30,30*30 ]
    for k in range(window[0], 
                   window[1], 1):
        # ensure that several time points in a row are below
        temp = data_in[k:k+n_vals_below_thresh]

        #if animal_id !='IA2':
        if np.all(temp<=(mean2-std*std_factor[ctr])):
            idx_out = k 
            break
        #else:
        #    if np.all(temp>=(mean2+std*std_factor[ctr])):
        #        idx_out = k 
        #        break

    #
    if idx_out>(900-30//2):
        idx_out=np.nan
        
    return idx_out

In [209]:
# ############################################ 
# ############################################ 
# ############################################ 
# animal_id = 'IA3'
# lockout = 15
# all_means, all_vars, names = load_variances(animal_id,
#                                        lockout)



In [210]:
####################### 
from scipy import signal

#
animal_id = 'AQ2'
lockout = 15
window_length = 50
all_means, all_vars, names = load_variances(animal_id,
                                            lockout,
                                            window_length)

# OLD ROI IDS
#roi_ids = [15,5,9,11,1]

# new ROI IDS
roi_ids = [15,10,12,13,8]

#
session_ids = np.arange(len(all_vars))
print ("session_ids: ", session_ids)

#
colors = plt.cm.viridis(np.linspace(0,1,len(all_vars)))

###############################################
###############################################
###############################################
plot_vars = False
fig = plt.figure(figsize=(20,5))
std_factor = [2,2,2,4,2]
n_vals = [10,15,20,20,20,30]
first_decay = []

# plot first decay point
s1 = 400
e1 = 600

#
for ctr, roi_id in enumerate(roi_ids):
    
    first_decay.append([])
    
    # 
    ax = plt.subplot(1,5,ctr+1)
    all_ = []
    for ctr_sess, session_id in enumerate(session_ids):
        if plot_vars:
            temp = all_vars[session_id][roi_id].copy()
        else:
            temp = all_means[session_id][:,roi_id].copy()
        
        print ("TEMP: ", temp.shape)
        print (all_vars[session_id].shape)
        #
        t = np.arange(temp.shape[0])/30.-30

        # 
        if True:
            temp = signal.savgol_filter(temp, 31, 2)

        elif False:
            fs = 30
            fc = 5  # Cut-off frequency of the filter

            w = fc / (fs / 2) # Normalize the frequency
            b, a = signal.butter(fc, w, 'low')
            temp = signal.filtfilt(b, a, temp)

        # 
        plt.plot(t, temp, 
                 color=colors[ctr_sess], 
                 alpha=.5,
                 linewidth=3)

        idx = find_first_variance_decrease_point2(temp, s1, e1, std_factor, ctr,
                                                 animal_id,
                                                 n_vals[ctr])

        all_.append(temp)

        if np.isnan(idx)==True:
            continue

        first_decay[ctr].append(t[idx])
        

    # 
    a = np.array(all_)
    
    if False:
        a = savgol_filter(a, 31, 2)
    
    a_mean = np.mean(a,axis=0)
    plt.plot(t, a_mean,c='red',
             linewidth=2,
             alpha=1)
    
    # 
    idx = find_first_variance_decrease_point2(a_mean, s1, e1,
                                             std_factor, ctr,
                                             animal_id, 
                                             n_vals[ctr])
    if np.isnan(idx)==True:
        evdt = 0
    else:
        evdt = t[idx]
    print ("EVDT: ", evdt, " sec", first_decay)
    #
    if False:
        plt.plot([t[idx+600], t[idx+601]],
             [a_mean[idx+600],a_mean[idx+601]],
               c='black',
               linewidth=10,
           alpha=1)
    
    plt.xlim(-15,5)
    
    plt.title(names[roi_id]+ " " + str(round(evdt,2)) + " sec" ,fontsize=8)
    
    # 
    if plot_vars:
        plt.ylim(bottom=0)
        #plt.ylim(bottom=0)
        plt.plot([-30,30],[0,0],'--',c='grey')
        plt.plot([0,0],[-.2,.2],'--',c='grey')
    else:
        plt.ylim(-75,100)
        plt.plot([-30,30],[0,0],'--',c='grey')
        plt.plot([0,0],[-75,100],'--',c='grey')
    #plt.title(saved_names[roi_id])
    
plt.suptitle(animal_id+ " # of sessions: "+str(len(session_ids)))
if True:
    plt.savefig('/home/cat/variance_analysis_var_lockout'+str(plot_vars)+'.svg')
    plt.close()
else:
    plt.show()

(594, 1800, 16)
temp:  (1800, 16)
temp:  (1800, 16)


<ipython-input-208-c24f92db84de>:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  locaNMF_temporal = np.array(temp_out)


temp:  (1800, 16)
temp:  (1800, 16)
temp:  (1800, 16)
temp:  (1800, 16)
temp:  (1800, 16)
temp:  (1800, 16)
temp:  (1800, 16)
temp:  (1800, 16)
temp:  (1800, 16)
session_ids:  [ 0  1  2  3  4  5  6  7  8  9 10]
TEMP:  (1800,)
(16, 1800)
TEMP:  (1800,)
(16, 1800)
TEMP:  (1800,)
(16, 1800)
TEMP:  (1800,)
(16, 1800)
TEMP:  (1800,)
(16, 1800)
TEMP:  (1800,)
(16, 1800)
TEMP:  (1800,)
(16, 1800)
TEMP:  (1800,)
(16, 1800)
TEMP:  (1800,)
(16, 1800)
TEMP:  (1800,)
(16, 1800)
TEMP:  (1800,)
(16, 1800)
EVDT:  -3.3000000000000007  sec [[-4.033333333333335, -3.866666666666667, -5.266666666666666, -3.133333333333333, -3.2333333333333343, -5.100000000000001, -3.3000000000000007, -3.2666666666666657]]
TEMP:  (1800,)
(16, 1800)
TEMP:  (1800,)
(16, 1800)
TEMP:  (1800,)
(16, 1800)
TEMP:  (1800,)
(16, 1800)
TEMP:  (1800,)
(16, 1800)
TEMP:  (1800,)
(16, 1800)
TEMP:  (1800,)
(16, 1800)
TEMP:  (1800,)
(16, 1800)
TEMP:  (1800,)
(16, 1800)
TEMP:  (1800,)
(16, 1800)
TEMP:  (1800,)
(16, 1800)
EVDT:  -1.966666666

In [163]:
print (len(all_vars))
print (all_vars[0].shape)

11
(1800,)


In [30]:
animal_id = 'IJ1'

#
data = np.load('/media/cat/4TBSSD/yuki/'+animal_id+'/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats.npz',
               allow_pickle=True)

trials = data['trials']
print (trials[0].shape)

trials = np.vstack(trials)
print (trials.shape)

plt.figure()
t= np.arange(trials.shape[1])/30-30
for ctr,id_ in enumerate(roi_ids):
    ax=plt.subplot(1,5,ctr+1)
    temp = trials[:,:,id_]
    temp = np.median(temp,0)
    temp = temp-np.median(temp)
    plt.plot(t,temp, label=roi[ctr])
    
    plt.ylim(-40,+150)
plt.legend()
plt.show()


(12, 1800, 16)
(261, 1800, 16)


In [13]:
data = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Mar31_30Hz/IA1pm_Mar31_30Hz_locanmf.npz',
               allow_pickle=True)

names = data['names']
for k in range(len(names)):
    print (k, names[k])

roi = ['motor','somatosensory','upper limb','visual','retrosplenial']
roi_ids = [8,10,12,13,15]

0 Retrosplenial area, dorsal part, layer 1 - right
1 Retrosplenial area, lateral agranular part, layer 1 - right
2 Primary visual area, layer 1 - right
3 Primary somatosensory area, upper limb, layer 1 - right
4 Primary somatosensory area, lower limb, layer 1 - right
5 Primary somatosensory area, barrel field, layer 1 - right
6 Secondary motor area, layer 1 - right
7 Primary motor area, Layer 1 - right
8 Primary motor area, Layer 1 - left
9 Secondary motor area, layer 1 - left
10 Primary somatosensory area, barrel field, layer 1 - left
11 Primary somatosensory area, lower limb, layer 1 - left
12 Primary somatosensory area, upper limb, layer 1 - left
13 Primary visual area, layer 1 - left
14 Retrosplenial area, lateral agranular part, layer 1 - left
15 Retrosplenial area, dorsal part, layer 1 - left
